In [ ]:
import praw
import pandas as pd
import datetime as dt
from IPython import display
import math
from pprint import pprint
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [ ]:
reddit = praw.Reddit(client_id='jgrP89BE9hlOpA', \
                     client_secret='PQwrREBiVDW-CSLhq6IBAqeXfn0', \
                     user_agent='PortlandCrime / Domestic Violence and Child Welfare in Portland, ME', \
                     username='GBarboza-336', \
                     password='aJhisde1')
headlines = set()

In [ ]:
#subreddit = reddit.subreddit('CPTSD')
subreddit = reddit.subreddit('socialwork')
for submission in reddit.subreddit('socialwork').new(limit=None):
    headlines.add(submission.title)
    display.clear_output()
    print(len(headlines))

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for line in headlines:
    cw_score = sia.polarity_scores(line)
    cw_score['headline'] = line
    results.append(cw_score)

pprint(results[:6], width=100)

In [ ]:
df = pd.DataFrame.from_records(results)
df.head()

In [ ]:
df['label'] = 0
df.loc[df['compound'] > 0.2, 'label'] = 1
df.loc[df['compound'] < -0.2, 'label'] = -1
df.head()

In [ ]:
df2 = df[['headline', 'label']]
df2.to_csv('CPTSD_headlines_labels1.csv', mode='a', encoding='utf-8', index=False)
df2.head()

In [ ]:
print("Positive headlines:\n")
pprint(list(df[df['label'] == 1].headline)[:5], width=200)

print("\nNegative headlines:\n")
pprint(list(df[df['label'] == -1].headline)[:5], width=200)

In [ ]:
print(df.label.value_counts())

print(df.label.value_counts(normalize=True) * 100)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

counts = df.label.value_counts(normalize=True) * 100

sns.barplot(x=counts.index, y=counts, ax=ax)

ax.set_xticklabels(['Negative', 'Neutral', 'Positive'])
ax.set_ylabel("Percentage")

plt.show()

In [ ]:
from nltk.tokenize import word_tokenize, RegexpTokenizer

example = "This is an example sentence! However, it isn't a very informative one"

print(word_tokenize(example, language='english'))

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize(example)

In [ ]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
print(stop_words[:20])

In [ ]:
def process_text(headlines):
    tokens = []
    for line in headlines:
        toks = tokenizer.tokenize(line)
        toks = [t.lower() for t in toks if t.lower() not in stop_words]
        tokens.extend(toks)
    
    return tokens

In [ ]:
pos_lines = list(df[df.label == 1].headline)

pos_tokens = process_text(pos_lines)
pos_freq = nltk.FreqDist(pos_tokens)

pos_freq.most_common(20)

In [ ]:
y_val = [x[1] for x in pos_freq.most_common()]

fig = plt.figure(figsize=(10,5))
plt.plot(y_val)

plt.xlabel("Words")
plt.ylabel("Frequency")
plt.title("Word Frequency Distribution (Positive)")
plt.show()

In [ ]:
y_final = []
for i, k, z, t in zip(y_val[0::4], y_val[1::4], y_val[2::4], y_val[3::4]):
    y_final.append(math.log(i + k + z + t))

x_val = [math.log(i + 1) for i in range(len(y_final))]

fig = plt.figure(figsize=(10,5))

plt.xlabel("Words (Log)")
plt.ylabel("Frequency (Log)")
plt.title("Word Frequency Distribution (Positive)")
plt.plot(x_val, y_final)
plt.show()

In [ ]:
neg_lines = list(df2[df2.label == -1].headline)

neg_tokens = process_text(neg_lines)
neg_freq = nltk.FreqDist(neg_tokens)

neg_freq.most_common(20)

In [ ]:
y_val = [x[1] for x in neg_freq.most_common()]

fig = plt.figure(figsize=(10,5))
plt.plot(y_val)

plt.xlabel("Words")
plt.ylabel("Frequency")
plt.title("Word Frequency Distribution (Negative)")
plt.show()

In [ ]:
y_final = []
for i, k, z in zip(y_val[0::3], y_val[1::3], y_val[2::3]):
    if i + k + z == 0:
        break
    y_final.append(math.log(i + k + z))

x_val = [math.log(i+1) for i in range(len(y_final))]

fig = plt.figure(figsize=(10,5))

plt.xlabel("Words (Log)")
plt.ylabel("Frequency (Log)")
plt.title("Word Frequency Distribution (Negative)")
plt.plot(x_val, y_final)
plt.show()

In [ ]:
import math
import random
from collections import defaultdict
from pprint import pprint

# Prevent future/deprecation warnings from showing in output
import warnings
warnings.filterwarnings(action='ignore')

# Set global styles for plots
sns.set_style(style='white')
sns.set_context(context='notebook', font_scale=1.3, rc={'figure.figsize': (16,9)})

In [ ]:
df = pd.read_csv('C:/Users/g.barboza/CPTSD_headlines_labels1.csv', encoding='utf-8')
df.head()

In [ ]:
df = df[df.label != 0]
df.label.value_counts()

In [ ]:
top_subreddit = subreddit.top()

In [ ]:
hot_subreddit = subreddit.hot(limit=1000)

In [ ]:
top_subreddit = subreddit.top(limit=1000)

In [ ]:
for submission in subreddit.top(limit=10):
    print(submission.title, submission.id)

In [ ]:
for submission in subreddit.hot(limit=10):
    print(submission.title, submission.id)

In [ ]:
topics_dict = { "title":[], "score":[], "id":[], "url":[], "comms_num": [], "created": [], "body":[]}

In [ ]:
for submission in top_subreddit:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)

In [ ]:
topics_data = pd.DataFrame(topics_dict)
topics_data.drop_duplicates(subset=['id'], inplace=True)
topics_data

In [ ]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [ ]:
_timestamp = topics_data["created"].apply(get_date)

In [ ]:
topics_data = topics_data.assign(timestamp = _timestamp)

In [ ]:
topics_data.to_csv('FILENAME.csv') 